# Lab 6: Softmax Classification - 소프트맥스 분류


## Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
#For reproducibiility
torch.manual_seed(1)

## Softmax

소프트맥스(Softmax)를 이용하여 숫자를 확률로 변환한다.
$$ P(class = i) = \cfrac{e^{y_i}}{\sum\limits_{j} e^{y_j}} $$

PyTorch는 `softmax`함수를 가지고 있다. `dim=`은 어떤 차원에 따라 소프트맥스를 적용할 것인지 결정한다.

In [3]:
z = torch.FloatTensor([1, 2, 3])

hypothesis = F.softmax(z, dim = 0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


소프트맥스의 결과는 확률이기 때문에, 그 합은 1이 된다.

In [4]:
hypothesis.sum()

tensor(1.)

## Cross Entropy Loss (Low-level)

다중 클래스 분류(multi-class classification)을 위해서, 교차 엔트로피(Cross Entropy) 오차를 이용한다.
  
$$ L = \cfrac{1}{N} \sum -y\log(\hat y) $$
  
$\hat y$는 예측된 가능성이고, $y$는 정확한 가능성(0 또는 1)이다.

In [5]:
z = torch.rand(3, 5, requires_grad= True)
hypothesis = F.softmax(z, dim= 1)
print(hypothesis)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward0>)


In [6]:
y = torch.randint(5, (3,)).long()
print(y)

tensor([0, 2, 1])


In [7]:
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1) #scatter에 대한 더 자세한 내용은 PyTorch_scatter.ipynb 참고

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [8]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim= 1).mean() #Cross Entropy Loss
print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)


## Cross Entropy Loss with `torch.nn.functional`

PyTorch는 `F.log_softmax()`를 가지고 있다. 이 함수의 출력은 소프트맥스를 적용한 결과에 로그를 취한 것이다.

In [9]:
#Low level
print(torch.log(F.softmax(z, dim= 1)))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward0>)


In [10]:
#High Level
print(F.log_softmax(z, dim= 1))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward0>)


PyTorch는 [`F.nll_loss()`](https://pytorch.org/docs/stable/generated/torch.nn.functional.nll_loss.html#torch.nn.functional.nll_loss)함수 또한 가지고 있는데, 이는 negative log likelihood loss를 나타낸다. 이 함수는 `F.log_softmax()`의 결과를 입력으로 받아서 교차 엔트로피 손실(Cross Entropy Loss)을 계산하는 것 같다.

In [11]:
#Low level
(y_one_hot * -torch.log(F.softmax(z, dim= 1))).sum(dim= 1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

In [12]:
#High level
F.nll_loss(F.log_softmax(z, dim= 1), y)

tensor(1.4689, grad_fn=<NllLossBackward0>)

PyTorch는 `F.cross_entropy`도 가지고 있는데, 이것은 `F.log_softmax()`와 `F.nll_loss()`를 결합한 것이다.

In [13]:
F.cross_entropy(z, y)

tensor(1.4689, grad_fn=<NllLossBackward0>)

## Training with Low-level Cross Entropy Loss

In [14]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]

x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [15]:
#모델 초기화
W = torch.zeros((4, 3), requires_grad= True)
b = torch.zeros(3, requires_grad= True) #원본에는 zeros의 첫번째 인자가 1이었지만, 3으로 고침

#optimizer 설정
optimizer = optim.SGD([W, b], lr= 0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
  #H(x) 계산
  hypothesis = F.softmax(x_train @ W + b, dim= 1)
  #Cost 계산 (Cross Entropy Loss)
  y_one_hot = torch.zeros_like(hypothesis).scatter(1, y_train.unsqueeze(1), 1)
  cost = (y_one_hot * -torch.log(hypothesis)).sum(dim= 1).mean()

  #H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  # 100번마다 로그 출력
  if epoch % 100 == 0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
      epoch, nb_epochs, cost.item()
    ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.704199
Epoch  200/1000 Cost: 0.622999
Epoch  300/1000 Cost: 0.565717
Epoch  400/1000 Cost: 0.515291
Epoch  500/1000 Cost: 0.467661
Epoch  600/1000 Cost: 0.421278
Epoch  700/1000 Cost: 0.375402
Epoch  800/1000 Cost: 0.329766
Epoch  900/1000 Cost: 0.285072
Epoch 1000/1000 Cost: 0.248155


## Training with `F.cross_entropy`

In [16]:
#모델 초기화
W = torch.zeros((4, 3), requires_grad= True)
b = torch.zeros(3, requires_grad= True) #원본에는 zeros의 첫번째 인자가 1이었지만, 3으로 고침

#optimizer 설정
optimizer = optim.SGD([W, b], lr= 0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
  #H(x) 계산
  z = x_train @ W + b
  #Cost 계산 (Cross Entropy Loss)
  cost = F.cross_entropy(z, y_train)

  #H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  # 100번마다 로그 출력
  if epoch % 100 == 0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
      epoch, nb_epochs, cost.item()
    ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.704199
Epoch  200/1000 Cost: 0.623000
Epoch  300/1000 Cost: 0.565717
Epoch  400/1000 Cost: 0.515291
Epoch  500/1000 Cost: 0.467662
Epoch  600/1000 Cost: 0.421278
Epoch  700/1000 Cost: 0.375402
Epoch  800/1000 Cost: 0.329766
Epoch  900/1000 Cost: 0.285073
Epoch 1000/1000 Cost: 0.248155


## High-level Implementation with `nn.Module`

In [17]:
class SoftmaxClassifierModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(4, 3) #입력 차원은 4, 출력 차원(레이블, 클래스 수)은 3
  
  def forward(self, x):
    return self.linear(x)

model = SoftmaxClassifierModel()

In [18]:
#optimizer 설정
optimizer = optim.SGD(model.parameters(), lr= 0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
  #H 계산
  prediction = model(x_train)
  #cost 계산 (Cross Entropy Loss)
  cost = F.cross_entropy(prediction, y_train)

  #H 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  #100번마다 로그 출력
  if epoch % 100 == 0:
    print('epoch: {:4d}/{}  Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

epoch:    0/1000  Cost: 1.849513
epoch:  100/1000  Cost: 0.689894
epoch:  200/1000  Cost: 0.609259
epoch:  300/1000  Cost: 0.551218
epoch:  400/1000  Cost: 0.500141
epoch:  500/1000  Cost: 0.451947
epoch:  600/1000  Cost: 0.405051
epoch:  700/1000  Cost: 0.358733
epoch:  800/1000  Cost: 0.312912
epoch:  900/1000  Cost: 0.269522
epoch: 1000/1000  Cost: 0.241922
